# HAR LSTM training 

In [1]:
# Imports
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

In [2]:
Xtrain, Ytrain, list_ch_train = read_data(data_path="/home/arasdar/datasets/har-data/", split="train") # train
Xtest, Ytest, list_ch_test = read_data(data_path="/home/arasdar/datasets/har-data/", split="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

/home/arasdar/github/deep-learning-HAR/utils/utilities.py:37: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  X[:,:,i_ch] = dat_.as_matrix()


In [3]:
# # Standardize
# X_train, X_test = standardize(X_train, X_test) 

Train/Validation Split

In [4]:
Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(Xtrain, Ytrain,
                                                  stratify=Ytrain, random_state=123)

One-hot encoding:

In [5]:
# y_tr = one_hot(lab_tr)
# y_vld = one_hot(lab_vld)
# y_test = one_hot(labels_test)

### Hyperparameters

In [6]:
# Imports
import tensorflow as tf

lstm_size = 27         # 3 times the amount of channels
lstm_layers = 2        # Number of layers
batch_size = 600       # Batch size
seq_len = 128          # Number of steps
learning_rate = 0.0001 # Learning rate (default is 0.001)
epochs = 1000

# Fixed
n_classes = 6
n_channels = 9

### Construct the graph
Placeholders

In [7]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    print(inputs_.shape, inputs_.dtype)
    indices_ = tf.placeholder(tf.int32, [None], name = 'indices')
    print(indices_.shape, indices_.dtype)
    #     keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    #     learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

(?, 128, 9) <dtype: 'float32'>
(?,) <dtype: 'int32'>


Construct inputs to LSTM

In [8]:
with graph.as_default():
    # Construct the LSTM inputs and LSTM cells
    lstm_in = tf.transpose(inputs_, [1,0,2]) # reshape into (seq_len, N, channels)
    print(lstm_in.shape, lstm_in.dtype)
    lstm_in = tf.reshape(lstm_in, [-1, n_channels]) # Now (seq_len*N, n_channels)
    print(lstm_in.shape, lstm_in.dtype)
    
    # To cells: Resize input data to the RNN input
    lstm_in = tf.layers.dense(lstm_in, lstm_size, activation=None) # or tf.nn.relu, tf.nn.sigmoid, tf.nn.tanh?
    print(lstm_in.shape, lstm_in.dtype)
    
    # Open up the tensor into a list of seq_len pieces
    lstm_in = tf.split(lstm_in, seq_len, 0)
    print(len(lstm_in), lstm_in[0].shape, lstm_in[0].dtype)
    
    # Add LSTM layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    #drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, final_state = tf.contrib.rnn.static_rnn(cell, lstm_in, dtype=tf.float32, initial_state = initial_state)
    print(len(outputs), outputs[-1])
    
    # To cells: Resize the RNN output to output classes
    # We only need the last output tensor to pass into a classifier: Resize to the output classes/labels
    logits = tf.layers.dense(inputs=outputs[-1], units=n_classes, name='logits')
    labels = tf.one_hot(indices=indices_, depth=n_classes, dtype=logits.dtype)
    print(logits.shape, logits.dtype)
    print(labels.shape, labels.dtype)

(128, ?, 9) <dtype: 'float32'>
(?, 9) <dtype: 'float32'>
(?, 27) <dtype: 'float32'>
128 (?, 27) <dtype: 'float32'>
128 Tensor("rnn/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/Mul_767:0", shape=(600, 27), dtype=float32)
(600, 6) <dtype: 'float32'>
(?, 6) <dtype: 'float32'>


In [15]:
initial_state[0][0].shape

TensorShape([Dimension(600), Dimension(27)])

Define forward pass, cost function and optimizer:

In [9]:
with graph.as_default():
    
    # Model loss using logits and labels
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    
    # Model opimizier/update
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost) # No grad clipping
    
    #     # Grad clipping
    #     train_op = tf.train.AdamOptimizer(learning_rate_)

    #     gradients = train_op.compute_gradients(cost)
    #     capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    #     optimizer = train_op.apply_gradients(capped_gradients)

    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [10]:
if (os.path.exists('checkpoints') == False):
    !mkdir checkpoints

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    
    for e in range(epochs):
        
        # Training: Loop over batches
        state = sess.run(initial_state)
        for Xbatch, Ybatch in get_batches(Xtrain, Ytrain, batch_size):
            
            # Feed dictionary
            feed_dict = {inputs_: Xbatch, indices_: Ybatch, initial_state: state}
            loss, _ , state, acc = sess.run([cost, optimizer, final_state, accuracy], feed_dict)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 25 iterations
            if (iteration%25 == 0):
                
                # Initiate for validation set
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                
                val_acc_ = []
                val_loss_ = []
                for Xbatch, Ybatch in get_batches(Xvalid, Yvalid, batch_size):
                    
                    # Feed dictionary
                    feed_dict = {inputs_: Xbatch, indices_: Ybatch, initial_state: val_state}
                    loss_v, state_v, acc_v = sess.run([cost, final_state, accuracy], feed_dict)
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints/har-lstm.ckpt")

Epoch: 0/1000 Iteration: 5 Train loss: 1.412533 Train acc: 0.173333
Epoch: 1/1000 Iteration: 10 Train loss: 1.439450 Train acc: 0.168333
Epoch: 1/1000 Iteration: 15 Train loss: 1.411583 Train acc: 0.198333
Epoch: 2/1000 Iteration: 20 Train loss: 1.422266 Train acc: 0.236667
Epoch: 2/1000 Iteration: 25 Train loss: 1.432818 Train acc: 0.305000
Epoch: 2/1000 Iteration: 25 Validation loss: 1.439249 Validation acc: 0.331111
Epoch: 3/1000 Iteration: 30 Train loss: 1.451677 Train acc: 0.358333
Epoch: 3/1000 Iteration: 35 Train loss: 1.462592 Train acc: 0.381667
Epoch: 4/1000 Iteration: 40 Train loss: 1.473713 Train acc: 0.380000
Epoch: 4/1000 Iteration: 45 Train loss: 1.485318 Train acc: 0.360000
Epoch: 5/1000 Iteration: 50 Train loss: 1.378803 Train acc: 0.390000
Epoch: 5/1000 Iteration: 50 Validation loss: 1.418521 Validation acc: 0.370000
Epoch: 6/1000 Iteration: 55 Train loss: 1.401256 Train acc: 0.385000
Epoch: 6/1000 Iteration: 60 Train loss: 1.373415 Train acc: 0.350000
Epoch: 7/1000 I

Epoch: 53/1000 Iteration: 480 Train loss: 0.752572 Train acc: 0.555000
Epoch: 53/1000 Iteration: 485 Train loss: 0.733710 Train acc: 0.581667
Epoch: 54/1000 Iteration: 490 Train loss: 0.756872 Train acc: 0.551667
Epoch: 54/1000 Iteration: 495 Train loss: 0.761436 Train acc: 0.556667
Epoch: 55/1000 Iteration: 500 Train loss: 0.704576 Train acc: 0.533333
Epoch: 55/1000 Iteration: 500 Validation loss: 0.719950 Validation acc: 0.542778
Epoch: 56/1000 Iteration: 505 Train loss: 0.707111 Train acc: 0.558333
Epoch: 56/1000 Iteration: 510 Train loss: 0.684617 Train acc: 0.553333
Epoch: 57/1000 Iteration: 515 Train loss: 0.707263 Train acc: 0.558333
Epoch: 57/1000 Iteration: 520 Train loss: 0.709509 Train acc: 0.570000
Epoch: 58/1000 Iteration: 525 Train loss: 0.719967 Train acc: 0.575000
Epoch: 58/1000 Iteration: 525 Validation loss: 0.704847 Validation acc: 0.563889
Epoch: 58/1000 Iteration: 530 Train loss: 0.706655 Train acc: 0.601667
Epoch: 59/1000 Iteration: 535 Train loss: 0.729057 Train 

Epoch: 105/1000 Iteration: 950 Validation loss: 0.505693 Validation acc: 0.663333
Epoch: 106/1000 Iteration: 955 Train loss: 0.465703 Train acc: 0.678333
Epoch: 106/1000 Iteration: 960 Train loss: 0.464561 Train acc: 0.665000
Epoch: 107/1000 Iteration: 965 Train loss: 0.501257 Train acc: 0.646667
Epoch: 107/1000 Iteration: 970 Train loss: 0.473570 Train acc: 0.671667
Epoch: 108/1000 Iteration: 975 Train loss: 0.509602 Train acc: 0.661667
Epoch: 108/1000 Iteration: 975 Validation loss: 0.501529 Validation acc: 0.666667
Epoch: 108/1000 Iteration: 980 Train loss: 0.482187 Train acc: 0.700000
Epoch: 109/1000 Iteration: 985 Train loss: 0.499026 Train acc: 0.681667
Epoch: 109/1000 Iteration: 990 Train loss: 0.517336 Train acc: 0.691667
Epoch: 110/1000 Iteration: 995 Train loss: 0.488738 Train acc: 0.635000
Epoch: 111/1000 Iteration: 1000 Train loss: 0.446551 Train acc: 0.695000
Epoch: 111/1000 Iteration: 1000 Validation loss: 0.489211 Validation acc: 0.675556
Epoch: 111/1000 Iteration: 1005 

Epoch: 156/1000 Iteration: 1410 Train loss: 0.373076 Train acc: 0.708333
Epoch: 157/1000 Iteration: 1415 Train loss: 0.403914 Train acc: 0.695000
Epoch: 157/1000 Iteration: 1420 Train loss: 0.385970 Train acc: 0.708333
Epoch: 158/1000 Iteration: 1425 Train loss: 0.413408 Train acc: 0.710000
Epoch: 158/1000 Iteration: 1425 Validation loss: 0.404849 Validation acc: 0.707778
Epoch: 158/1000 Iteration: 1430 Train loss: 0.407553 Train acc: 0.726667
Epoch: 159/1000 Iteration: 1435 Train loss: 0.410987 Train acc: 0.716667
Epoch: 159/1000 Iteration: 1440 Train loss: 0.425427 Train acc: 0.726667
Epoch: 160/1000 Iteration: 1445 Train loss: 0.395495 Train acc: 0.675000
Epoch: 161/1000 Iteration: 1450 Train loss: 0.356347 Train acc: 0.728333
Epoch: 161/1000 Iteration: 1450 Validation loss: 0.396364 Validation acc: 0.710556
Epoch: 161/1000 Iteration: 1455 Train loss: 0.366114 Train acc: 0.705000
Epoch: 162/1000 Iteration: 1460 Train loss: 0.389974 Train acc: 0.700000
Epoch: 162/1000 Iteration: 1465

Epoch: 207/1000 Iteration: 1870 Train loss: 0.312468 Train acc: 0.736667
Epoch: 208/1000 Iteration: 1875 Train loss: 0.337829 Train acc: 0.733333
Epoch: 208/1000 Iteration: 1875 Validation loss: 0.338833 Validation acc: 0.733889
Epoch: 208/1000 Iteration: 1880 Train loss: 0.323589 Train acc: 0.761667
Epoch: 209/1000 Iteration: 1885 Train loss: 0.328570 Train acc: 0.750000
Epoch: 209/1000 Iteration: 1890 Train loss: 0.349604 Train acc: 0.755000
Epoch: 210/1000 Iteration: 1895 Train loss: 0.321634 Train acc: 0.703333
Epoch: 211/1000 Iteration: 1900 Train loss: 0.293681 Train acc: 0.740000
Epoch: 211/1000 Iteration: 1900 Validation loss: 0.335942 Validation acc: 0.736667
Epoch: 211/1000 Iteration: 1905 Train loss: 0.294487 Train acc: 0.723333
Epoch: 212/1000 Iteration: 1910 Train loss: 0.313740 Train acc: 0.723333
Epoch: 212/1000 Iteration: 1915 Train loss: 0.306263 Train acc: 0.733333
Epoch: 213/1000 Iteration: 1920 Train loss: 0.325917 Train acc: 0.740000
Epoch: 213/1000 Iteration: 1925

Epoch: 258/1000 Iteration: 2325 Validation loss: 0.335449 Validation acc: 0.717222
Epoch: 258/1000 Iteration: 2330 Train loss: 0.304355 Train acc: 0.756667
Epoch: 259/1000 Iteration: 2335 Train loss: 0.319461 Train acc: 0.748333
Epoch: 259/1000 Iteration: 2340 Train loss: 0.327637 Train acc: 0.756667
Epoch: 260/1000 Iteration: 2345 Train loss: 0.300873 Train acc: 0.706667
Epoch: 261/1000 Iteration: 2350 Train loss: 0.262688 Train acc: 0.745000
Epoch: 261/1000 Iteration: 2350 Validation loss: 0.306580 Validation acc: 0.738333
Epoch: 261/1000 Iteration: 2355 Train loss: 0.278141 Train acc: 0.723333
Epoch: 262/1000 Iteration: 2360 Train loss: 0.290310 Train acc: 0.728333
Epoch: 262/1000 Iteration: 2365 Train loss: 0.271936 Train acc: 0.745000
Epoch: 263/1000 Iteration: 2370 Train loss: 0.296841 Train acc: 0.741667
Epoch: 263/1000 Iteration: 2375 Train loss: 0.296847 Train acc: 0.758333
Epoch: 263/1000 Iteration: 2375 Validation loss: 0.308694 Validation acc: 0.740556
Epoch: 264/1000 Itera

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 25 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 25 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Testing: Loop over bacthes
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for Xbatch, Ybatch in get_batches(Xtest, Ytest, batch_size):
        feed = {inputs_: Xbatch, indices_: Ybatch, initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))